# Web Scraping of Companies in "Welcome to the Jungle"

Web scraping of all the companies at Welcome to the Jungle website, their characteristics and number of positions open.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# Initializing the dataframe to store the companies database:
companies = pd.DataFrame()

In [48]:
def get_data():
    # Initializing the dataframe to store the companies database:
    companies = pd.DataFrame()
    
    # Basic fixed variables for the response
    url = "https://csekhvms53-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20vanilla%20JavaScript%20(lite)%203.24.12%3Breact-instantsearch%205.3.2%3BJS%20Helper%20(2.28.1)&x-algolia-application-id=CSEKHVMS53&x-algolia-api-key=02f0d440abc99cae37e126886438b266"
    text = """accept: application/json
    Accept-Encoding: gzip, deflate, br
    Accept-Language: en-GB,en-US;q=0.9,en;q=0.8
    Cache-Control: no-cache
    Connection: keep-alive
    Content-Length: 665
    content-type: application/x-www-form-urlencoded
    Host: csekhvms53-dsn.algolia.net
    Origin: https://www.welcometothejungle.com
    Pragma: no-cache
    Referer: https://www.welcometothejungle.com/en/companies?page=1
    Sec-Fetch-Dest: empty
    Sec-Fetch-Mode: cors
    Sec-Fetch-Site: cross-site
    User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"""
    headers = {h[0]:h[1] for x in text.split('\n') if (h := x.split(': '))}
    
    # Variable necessary because of the POST request and also the variable that rules over the pagination:
    payload = """{"requests":[{"indexName":"wk_cms_organizations_production","params":"query=&hitsPerPage=30&maxValuesPerFacet=100&page=0&restrictSearchableAttributes=%5B%22name%22%2C%22alternative_names%22%2C%22offices.city%22%2C%22offices.state%22%2C%22offices.country%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%2C%22tools_name%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&filters=website.reference%3Awttj_fr&facets=%5B%22offices.country_code%22%2C%22offices.state%22%2C%22offices.district%22%2C%22offices.location%22%2C%22online%22%2C%22sectors.parent.en%22%2C%22size.en%22%2C%22languages%22%5D&tagFilters=&facetFilters=%5B%22offices.country_code%3AFR%22%5D"}]}"""
    
    # Connecting to the website and gettign the first results (on the first page)
    response = requests.post(url, headers=headers, data=payload)
    data = response.json()["results"][0]["hits"]

    # Agregating the first rows of data in the dataframe:
    companies = pd.concat([companies,pd.json_normalize(data)])

    # Calculating the maximum number of pages for scrapping
    totalpages = int(np.ceil(response.json()["results"][0]["nbPages"]))
    totalpages

    # Looping through pages 2 until the end of the results requests:
    for i in range(totalpages-1):
        
        # Changing the payload variable to adjust for pagination and updating the "response" and "data" variables to get the next page:
        payload = """{"requests":[{"indexName":"wk_cms_organizations_production","params":"query=&hitsPerPage=30&maxValuesPerFacet=100&page="""+str(i+1)+"""&restrictSearchableAttributes=%5B%22name%22%2C%22alternative_names%22%2C%22offices.city%22%2C%22offices.state%22%2C%22offices.country%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%2C%22tools_name%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&filters=website.reference%3Awttj_fr&facets=%5B%22offices.country_code%22%2C%22offices.state%22%2C%22offices.district%22%2C%22offices.location%22%2C%22online%22%2C%22sectors.parent.en%22%2C%22size.en%22%2C%22languages%22%5D&tagFilters=&facetFilters=%5B%22offices.country_code%3AFR%22%5D"}]}"""
        response = requests.post(url, headers=headers, data=payload)
        data = response.json()["results"][0]["hits"]

        # Agregating data in the dataframe created for this finality
        companies = pd.concat([companies,pd.json_normalize(data)])
    
    # Resetting index so it would be representative of the Company ID
    companies.set_index("reference", inplace=True)
    
    return companies

,languages,name,alternative_names,reference,slug,jobs_count,sectors,labels,nb_employees,offices,...,_highlightResult.sectors_name.cs,_highlightResult.sectors_name.sk,_highlightResult.offices,_highlightResult.tools,_highlightResult.tools_name,cover_image.fr,cover_image.en,cover_image.es,cover_image.cs,cover_image.sk
0,[fr],Tomcat Factory,,aV9p0GJ,tomcat-factory,2,"[{'name': {'fr': 'Incubateur / Accélérateur', ...",[],7.0,"[{'city': 'Levallois-Perret', 'country': 'Fran...",...,[{'Sektor podnikových služeb': {'value': 'Incu...,[{'Sektor podnikových služieb': {'value': 'Ink...,"[{'city': {'value': 'Levallois-Perret', 'match...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[fr],MyUnisoft,,ZJ7ozLL,myunisoft,2,"[{'name': {'fr': 'Expertise comptable', 'en': ...",[],36.0,"[{'city': 'Massy', 'country': 'France', 'count...",...,[{'Poradenství a audit': {'value': 'Účetnictví...,[{'Poradenstvo a audit': {'value': 'Účtovníctv...,"[{'city': {'value': 'Massy', 'matchLevel': 'no...","[{'backend': {'value': 'Node.js', 'matchLevel'...","[{'backend': {'value': 'Node.js', 'matchLevel'...",NaN,NaN,NaN,NaN,NaN
2,"[en, fr]",Quable,,NZ8pzZ3,quable,5,[{'name': {'fr': 'Digital Marketing / Data Mar...,[],25.0,"[{'city': 'Aix-En-Provence', 'country': 'Franc...",...,[{'Poradenství a audit': {'value': 'Digital Ma...,[{'Poradenstvo a audit': {'value': 'Digitálny ...,"[{'city': {'value': 'Aix-En-Provence', 'matchL...","[{'backend': {'value': 'PHP', 'matchLevel': 'n...","[{'backend': {'value': 'PHP', 'matchLevel': 'n...",NaN,NaN,NaN,NaN,NaN
3,"[en, fr]",Innovafeed,,46V9oyQ,innovafeed-1,33,[{'name': {'fr': 'Agroalimentaire / Nutrition ...,[],150.0,"[{'city': 'Decatur', 'country': 'United States...",...,[{'Průmysl / Zemědělství': {'value': 'Krmiva /...,[{'Priemysel': {'value': 'Krmivá / Výživa zvie...,"[{'city': {'value': 'Decatur', 'matchLevel': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[fr],Ubitransport,,j8K4Yez,ubitransport,15,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",[],110.0,"[{'city': 'Besançon', 'country': 'France', 'co...",...,"[{'Tech': {'value': 'Software', 'matchLevel': ...","[{'Technológie': {'value': 'Softvér', 'matchLe...","[{'city': {'value': 'Besançon', 'matchLevel': ...","[{'backend': {'value': 'PHP', 'matchLevel': 'n...","[{'backend': {'value': 'PHP', 'matchLevel': 'n...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,[fr],Isifid,,MbOxJeQ,isifid,0,"[{'name': {'fr': 'FinTech / InsurTech', 'en': ...",[],13.0,"[{'city': 'Paris', 'country': 'France', 'count...",...,[{'Bankovnictví / Pojišťovnictví / Finance': {...,[{'Bankovníctvo / Poisťovníctvo / Financie': {...,"[{'city': {'value': 'Paris', 'matchLevel': 'no...","[{'backend': {'value': 'PostgreSQL', 'matchLev...","[{'backend': {'value': 'PostgreSQL', 'matchLev...",NaN,NaN,NaN,NaN,NaN
6,[fr],SEKOIA,,D9z7WGl,sekoia,6,[{'name': {'fr': 'Intelligence artificielle / ...,[],85.0,"[{'city': 'Paris', 'country': 'France', 'count...",...,[{'Tech': {'value': 'Artificial Intelligence /...,[{'Technológie': {'value': 'Umelá inteligencia...,"[{'city': {'value': 'Paris', 'matchLevel': 'no...","[{'backend': {'value': 'Node.js', 'matchLevel'...","[{'backend': {'value': 'Node.js', 'matchLevel'...",NaN,NaN,NaN,NaN,NaN
7,[fr],DriiveMe,,LqzY4Q,driiveme,4,"[{'name': {'fr': 'Logistique', 'en': 'Logistic...",[],30.0,"[{'city': 'Paris', 'country': 'France', 'count...",...,[{'Mobilita / Transport': {'value': 'Logistika...,"[{'Mobilita / Doprava': {'value': 'Logistika',...","[{'city': {'value': 'Paris', 'matchLevel': 'no...","[{'frontend': {'value': 'Bootstrap', 'matchLev...","[{'frontend': {'value': 'Bootstrap', 'matchLev...",NaN,NaN,NaN,NaN,NaN
8,"[en, fr]",VO2 GROUP,,OPmkMmj,vo2-group,7,"[{'name': {'fr': 'IT / Digital', 'en': 'IT / D...",NaN,300.0,"[{'city': 'Casablanca', 'country': 'Morocco', ...",...,[{'Poradenství a audit': {'value': 'IT / Digit...,[{'Poradenstvo a audit': {'value': 'IT / Digit...,"[{'city': {'value': 'Casablanca', 'matchLevel'...","[{'backend': {'val

In [50]:
col = [c for c in companies.columns]
print(col)

['languages', 'name', 'alternative_names', 'reference', 'slug', 'jobs_count', 'sectors', 'labels', 'nb_employees', 'offices', 'tools', 'tools_name', 'published_at', 'has_tech_page', '_geoloc', 'objectID', 'website.reference', 'size.fr', 'size.en', 'size.es', 'size.cs', 'size.sk', 'sectors_name.fr', 'sectors_name.en', 'sectors_name.es', 'sectors_name.cs', 'sectors_name.sk', 'cover_image.fr.url', 'cover_image.fr.large.url', 'cover_image.fr.thumb.url', 'cover_image.fr.small.url', 'cover_image.en.url', 'cover_image.en.large.url', 'cover_image.en.thumb.url', 'cover_image.en.small.url', 'cover_image.es.url', 'cover_image.es.large.url', 'cover_image.es.thumb.url', 'cover_image.es.small.url', 'cover_image.cs.url', 'cover_image.cs.large.url', 'cover_image.cs.thumb.url', 'cover_image.cs.small.url', 'cover_image.sk.url', 'cover_image.sk.large.url', 'cover_image.sk.thumb.url', 'cover_image.sk.small.url', 'logo.url', 'logo.thumb.url', '_highlightResult.name.value', '_highlightResult.name.matchLevel

In [79]:
def drop_columns(df):
    
    # Droping columns that present all NaN values:
    df.dropna(axis=1, how="all", inplace=True)
    
    # Initializing the variable that will store the columns' names to be droped
    columns_to_drop=[]
    
    # Droping all columns that present at least 80% of NaN values:
    limit = 0.3 * df.shape[0]
    
    for col in df.columns:
        if df[col].isna().sum()>=limit:
            columns_to_drop += col
    
    # Droping columns that are not in English language (".en")
    for language in [".cs", ".es", ".fr", ".sk"]:
        columns_to_drop += [c for c in df.columns if language in c]
        
    # Droping from all useless columns that were manually identified as useless
    for col_name in ["_highlightResult.", "_image.", "logo.", 'alternative_names', 'labels']:
        columns_to_drop += [c for c in df.columns if col_name in c]
    
    # Cleaning all duplicates from columns list:
    columns_to_drop = list(set(columns_to_drop))
    
    # Finally, drop all columns previously selected
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

In [83]:
# Testing the effect of droping columns functions
companies_test = companies.copy()
print(companies_test.shape)
companies_test = clean_columns(companies_test)
print(companies_test.shape)
print(companies_test.columns)

(1000, 72)
(1000, 17)
languages


In [84]:
print(companies_test.columns)

Index(['languages', 'name', 'reference', 'slug', 'jobs_count', 'sectors',
       'nb_employees', 'offices', 'tools', 'tools_name', 'published_at',
       'has_tech_page', '_geoloc', 'objectID', 'website.reference', 'size.en',
       'sectors_name.en'],
      dtype='object')


In [85]:
companies_test

,languages,name,reference,slug,jobs_count,sectors,nb_employees,offices,tools,tools_name,published_at,has_tech_page,_geoloc,objectID,website.reference,size.en,sectors_name.en
0,[fr],Tomcat Factory,aV9p0GJ,tomcat-factory,2,"[{'name': {'fr': 'Incubateur / Accélérateur', ...",7.0,"[{'city': 'Levallois-Perret', 'country': 'Fran...",[],[],2021-02-19T15:02:44.511+01:00,False,"[{'lat': 48.88737, 'lng': 2.28544}]",10944,wttj_fr,< 15 employees,[{'Corporate Services': 'Incubator / Accelerat...
1,[fr],MyUnisoft,ZJ7ozLL,myunisoft,2,"[{'name': {'fr': 'Expertise comptable', 'en': ...",36.0,"[{'city': 'Massy', 'country': 'France', 'count...","[{'parent': 'backend', 'backend': 'Node.js', '...","[{'backend': 'Node.js'}, {'frontend': 'React J...",2021-02-19T11:19:39.294+01:00,True,"[{'lat': 48.72308, 'lng': 2.27037}]",10980,wttj_fr,Between 15 and 50 employees,"[{'Consulting / Audit': 'Accounting'}, {'Banki..."
2,"[en, fr]",Quable,NZ8pzZ3,quable,5,[{'name': {'fr': 'Digital Marketing / Data Mar...,25.0,"[{'city': 'Aix-En-Provence', 'country': 'Franc...","[{'parent': 'backend', 'backend': 'PHP', 'imag...","[{'backend': 'PHP'}, {'frontend': 'JavaScript'...",2021-02-19T10:01:10.270+01:00,True,"[{'lat': 43.52909, 'lng': 5.41105}, {'lat': 44...",944,wttj_fr,Between 15 and 50 employees,[{'Consulting / Audit': 'Digital Marketing / D...
3,"[en, fr]",Innovafeed,46V9oyQ,innovafeed-1,33,[{'name': {'fr': 'Agroalimentaire / Nutrition ...,150.0,"[{'city': 'Decatur', 'country': 'United States...",[],[],2021-02-18T19:52:20.567+01:00,False,"[{'lat': 33.77357, 'lng': -84.29745}, {'lat': ...",9461,wttj_fr,Between 50 and 250 employees,[{'Industry': 'Agri-food / Animal Nutrition'}]
4,[fr],Ubitransport,j8K4Yez,ubitransport,15,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",110.0,"[{'city': 'Besançon', 'country': 'France', 'co...","[{'parent': 'backend', 'backend': 'PHP', 'imag...","[{'backend': 'PHP'}, {'frontend': 'JavaScript'...",2021-02-18T16:40:13.817+01:00,True,"[{'lat': 47.2389, 'lng': 6.01377}, {'lat': 43....",6059,wttj_fr,Between 50 and 250 employees,"[{'Tech': 'Software'}, {'Mobility / Transport'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,[fr],Isifid,MbOxJeQ,isifid,0,"[{'name': {'fr': 'FinTech / InsurTech', 'en': ...",13.0,"[{'city': 'Paris', 'country': 'France', 'count...","[{'parent': 'backend', 'backend': 'PostgreSQL'...","[{'backend': 'PostgreSQL'}, {'backend': 'Sprin...",2020-02-13T10:58:26.761+01:00,True,"[{'lat': 48.87194, 'lng': 2.34853}, {'lat': 48...",7538,wttj_fr,< 15 employees,[{'Banking / Insurance / Finance': 'FinTech / ...
6,[fr],SEKOIA,D9z7WGl,sekoia,6,[{'name': {'fr': 'Intelligence artificielle / ...,85.0,"[{'city': 'Paris', 'country': 'France', 'count...","[{'parent': 'backend', 'backend': 'Node.js', '...","[{'backend': 'Node.js'}, {'frontend': 'Angular...",2020-02-13T10:18:15.126+01:00,True,"[{'lat': 48.8700955, 'lng': 2.3256528}, {'lat'...",2536,wttj_fr,Between 50 and 250 employees,[{'Tech': 'Artificial Intelligence / Machine L...
7,[fr],DriiveMe,LqzY4Q,driiveme,4,"[{'name': {'fr': 'Logistique', 'en': 'Logistic...",30.0,"[{'city': 'Paris', 'country': 'France', 'count...","[{'parent': 'frontend', 'frontend': 'Bootstrap...","[{'frontend': 'Bootstrap'}, {'backend': 'PHP'}...",2020-02-13T09:43:16.404+01:00,True,"[{'lat': 48.88473, 'lng': 2.31925}, {'lat': 48...",7565,wttj_fr,Between 15 and 50 employees,[{'Mobility / Transport': 'Logistics'}]
8,"[en, fr]",VO2 GROUP,OPmkMmj,vo2-group,7,"[{'name': {'fr': 'IT / Digital', 'en': 'IT / D...",300.0,"[{'city': 'Casablanca', 'country': 'Morocco', ...","[{'parent': 'backend', 'backend': 'Node.js', '...","[{'backend': 'Node.js'}, {'frontend': 'Angular...",2020-02-12T17:45:45.558+01:00,True,"[{'lat': 33.59662, 'lng': -7.61889}, {'lat': 4...",1981,wttj_fr,"Between 250 and 2,000 employees","[{'Consulting / Audit': 'IT / Digital'}, {'Con..."


In [77]:
companies_test[companies_test["slug"]=="renault-digital"]

,languages,name,alternative_names,reference,slug,jobs_count,sectors,labels,nb_employees,offices,tools,tools_name,published_at,has_tech_page,_geoloc,objectID,website.reference,size.en,sectors_name.en
3,[fr],Renault Digital,,aW0mYX7,renault-digital,0,"[{'name': {'fr': 'Application mobile', 'en': '...",[],200.0,"[{'city': 'Boulogne-Billancourt', 'country': '...","[{'parent': 'backend', 'backend': 'Node.js', '...","[{'backend': 'Node.js'}, {'frontend': 'Angular...",2020-02-13T15:10:21.838+01:00,True,"[{'lat': 48.8306909, 'lng': 2.2320178}]",4876,wttj_fr,Between 50 and 250 employees,"[{'Tech': 'Mobile Apps'}, {'Tech': 'Artificial..."


In [78]:
companies_test[companies_test["reference"]=="aqRkp77"]

,languages,name,alternative_names,reference,slug,jobs_count,sectors,labels,nb_employees,offices,tools,tools_name,published_at,has_tech_page,_geoloc,objectID,website.reference,size.en,sectors_name.en


In [ ]:
companies_test[companies_test["reference"]=="aqRkp77"]

In [ ]:
def unpack_lists():
    
    # Unpacking languages column 
    for i in range(len(companies.columns)):
        if type(companies_test.iloc[1,0])=="list":
            for element in 
            

In [109]:
print(companies_test.iloc[1,0])
type(companies_test.iloc[1,0])

['fr']


list